In [1]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Mounted at /content/drive


## Installation

In [9]:
!pip install -qU google-api-python-client langchain-openai langchain langchainhub langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00


In [35]:
import googleapiclient.discovery

def get_youtube_channel_thumbnails(api_key, channel_id):
  youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=api_key)
  channel_response = youtube.channels().list(
      part='contentDetails',
      id=channel_id
  ).execute()
  uploads_playlist_id = channel_response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

  # Getting Youtube Response
  next_page_token = None
  videos = []
  playlist_response = youtube.playlistItems().list(
      part='snippet',
      playlistId=uploads_playlist_id,
      maxResults=4,
      pageToken=next_page_token
  ).execute()

  # Getting Info
  for item in playlist_response["items"]:
    snippet = item["snippet"]
    # title
    video_title = snippet["title"]
    # description
    video_desc = snippet['description']
    # thumbnails - urls
    video_thumbnail_url = snippet['thumbnails']['standard']['url']
    videos.append({
        "title": video_title,
        "description": video_desc,
        "thumbnail_url": video_thumbnail_url
    })

  return videos

In [37]:
api_key = userdata.get('youtube_api_key_01')
# 살색의 박감독: UCuypTxyCwP2QCxXHOUuibCg
# 육식맨: UC0VR2v4TZeGcOrZHnmwbU_Q
appropriate_videos = get_youtube_channel_thumbnails(api_key, "UC0VR2v4TZeGcOrZHnmwbU_Q")
inappropriate_videos = get_youtube_channel_thumbnails(api_key, "UCuypTxyCwP2QCxXHOUuibCg")
print(appropriate_videos)
print(inappropriate_videos)

[{'title': '시칠리아 내장 특집 : 낯선 버거에게서 모란시장의 향기가 난다', 'description': '시칠리아 3부작을 사랑해주신 시청자 여러분, 모두 고개 숙여 감사드립니다!\n\n🔽오늘 영상에 등장한 장소 지도 링크와 촬영도구🔽\n✅ 브이로그 카메라 : https://link.coupang.com/a/bmCkbz\n✅ 푸드 카메라 : https://link.coupang.com/a/bmCliz\n✅ 동시녹음 보이스 레코더 : https://link.coupang.com/a/bKOfpo\n✅ 후시녹음 나레이션 마이크 : https://link.coupang.com/a/bnDkq7\n✅ 랩탑 : https://link.coupang.com/a/bmClTW\n(이상 제휴 링크로 구매 시 수수료를 받아 채널 운영에 도움이 됩니다.)\n\n스푸파 내장 버거 Porta Carbone https://maps.app.goo.gl/SbXNN2NgaocR1fUU7\n스푸파 아란치니 Bar Touring https://maps.app.goo.gl/4uYbRFZy5WZmUXgy6\n이탈리아 푸드 매거진 만점 젤라토 Cappadonia Gelati https://maps.app.goo.gl/hRwczjJUjw4MunkF6\n시칠리아 대표 에스프레소 바 Bar Del Corso https://maps.app.goo.gl/ysBxHjgXgM85c7zb8\n가장 감동한 팔레르모 로컬 식당 Antichi Sapori Palermitani https://maps.app.goo.gl/rBWwhPA3b8iYZsun9\n\n\n육식맨 (홈쿠킹&푸드트립) : https://www.youtube.com/@YOOXICMAN\n잡식맨 (일상&라이브&쇼츠) : https://www.youtube.com/@JOBXICMAN\n인스타그램 : https://www.instagram.com/yooxicman/\n비즈니스 메일 : yooxicman@gmail.com\n\n[육식맨 YOOXICMAN

In [38]:
# Seperating title, description and imgae urls
appropriate_text_description = []
appropriate_image_urls = []
for video in appropriate_videos:
  appropriate_text_description.append({"title": video['title'], "description":video['description']})
  appropriate_image_urls.append(video["thumbnail_url"])

inappropriate_text_description = []
inappropriate_image_urls = []
for video in inappropriate_videos:
  inappropriate_text_description.append({"title": video['title'], "description":video['description']})
  inappropriate_image_urls.append(video["thumbnail_url"])

## Text Evaluation

In [53]:
# Text: title & description
import os
import json

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

os.environ['OPENAI_API_KEY'] = userdata.get('openAI')

text_evaluation_prompt = """
You are a social media expert, specialized in detecting innapropriate, harmful or dangerous Youtube Channel.
You'll be given a list of four recent videos of a Youtube channel.
Each element of the list contains title and description about the video.
1. "title": "title of the video",
2. "description": "description of the video"
Based on the given information you need to judge whether the Youtube channel posted the given information is appropriate or not.

After the evaluation you only generate as follow:
{{
  "appropriate":"true or false"
  "reason":"empty if tis appropriate, explain why otherwise in Korean and one sentence"
}}

Here is the list of videos:
{list}
"""

def recent_video_text_evaluation(description):
  llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.7)
  prompt_template = ChatPromptTemplate.from_messages(
    [("system", text_evaluation_prompt), ("user", "{list}")]
  )
  chain = prompt_template | llm | StrOutputParser()
  return json.loads(chain.invoke({"list": description}))

In [54]:
print(recent_video_text_evaluation(appropriate_text_description))
print(recent_video_text_evaluation(inappropriate_text_description))

{'appropriate': 'true', 'reason': ''}
{'appropriate': 'false', 'reason': '이 채널의 콘텐츠는 성적으로 노골적이며 부적절한 내용을 포함하고 있어, 시청자에게 해롭다고 판단됩니다.'}


## Image Evaluation

In [57]:
from PIL import Image
import requests
from io import BytesIO

response = requests.get(appropriate_image_urls[0])
img = Image.open(BytesIO(response.content))

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7AD2208F9570>
